# Aggregate data in PostGIS

```sql
SELECT zh_districts.stzname, SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS total
FROM zh_districts, zh_roads
WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
GROUP BY zh_districts.stzname
```

```sql
SELECT zh_districts.stzname, SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS total
FROM zh_districts, zh_roads
WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
AND zh_roads.velo=1
GROUP BY zh_districts.stzname
```

```sql
CREATE VIEW v_bike_indicator AS
SELECT 
    zh_districts.stznr AS zonennr, 
    zh_districts.stzname AS zonenname, 
    velo_sum.total AS sum_velo, 
    total_sum.total AS sum_total,
    ROUND((velo_sum.total/total_sum.total*100)::numeric, 0) AS percentage,
    zh_districts.geom AS geom
FROM
    (SELECT zh_districts.fid, SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS total
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
    GROUP BY zh_districts.fid) AS total_sum,
    (SELECT zh_districts.fid, SUM(ST_Length(ST_Intersection(zh_districts.geom,zh_roads.geom))) AS total
    FROM zh_districts, zh_roads
    WHERE ST_Intersects(zh_districts.geom, zh_roads.geom)
    AND zh_roads.velo=1
    GROUP BY zh_districts.fid) AS velo_sum,
    zh_districts
WHERE velo_sum.fid=total_sum.fid AND total_sum.fid=zh_districts.fid;
```